In [ ]:
# Importar e tratar as bases de dados
# Importando bibliotecas e base de dados
import pandas as pd
import pathlib as pl
import win32com.client as win32

emails = pd.read_excel(r"Emails.xlsx")
# Lembrando o encoding foi utilizado para caracteres que podem estar contidos na tabela lojas
# O sep serve para organizar a tabela 
lojas = pd.read_csv(r"Lojas.csv", encoding='latin1', sep=';') 
vendas = pd.read_excel(r"Vendas.xlsx")
display(emails)
display(lojas)
display(vendas)


vendas = vendas.merge(lojas, on='ID Loja') # Ao mesclar as tabelas, agrupamos elas pelo ID Loja (Coluna)
display(vendas)

In [ ]:
# Criar um arquivo (DataFrame) para cada loja
dicionarios_lojas  = {} # Criando um dicionário vázio
for loja in lojas['Loja']: # Vou percorrer DF (tabela) lojas, na minha DF (tabela lojas) coluna Loja
    # Vou filtrar na coluna cada loja do meu DF e minhas linhas sendo infinitamente preenchidas enquanto haver lojas (são 25 lojas)
    dicionarios_lojas[loja] = vendas.loc[vendas['Loja']==loja, :] 

# display(dicionarios_lojas['Rio Mar Recife'])



In [ ]:
# Selecionando minhas maiores datas
dia_indicador = vendas['Data'].max() # Pegando minha maior data
print(dia_indicador)
print(f'{dia_indicador.day}/{dia_indicador.month}') # Selecionando só o dia e mês desta maior data

In [ ]:
# Salvar o Backup nas pastas
backup = pl.Path(r'C:\Users\ulyss\Downloads\Backup_automações-de-processo')

arquivos_backup = backup.iterdir() # Pega todos arquivos/pastas que estão dentro daquele determinado caminho (iterdir)

# List Compreehesion (a intenção é verficar se existe alguma pasta criada (arquivo))
lista_backup = [arquivo.name for arquivo in arquivos_backup] 

# print(lista_backup)

for loja in dicionarios_lojas:
    if loja not in lista_backup:
        nova_pasta = backup / loja # A junção do caminho só é permitida com (/), pois foi utilizado Pathlib
        nova_pasta.mkdir() # Foi criado minhas pastas para cada loja
        
        
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx' # Nome do arquivo
local_arquivo = backup / loja / nome_arquivo # O local onde o arquivo em excel será criado

# Será criado um arquivo em Excel para cada loja (Start) - Com as informações dos produtos de cada Loja
dicionarios_lojas[loja].to_excel(local_arquivo) 

# print(len(dicionarios_lojas))

In [ ]:
#definição de metas

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [ ]:
# Calcular o indicador para uma loja
loja = "Norte Shopping"
vendas_loja_ano = dicionarios_lojas[loja]
vendas_loja_dia = vendas_loja_ano.loc[vendas_loja_ano['Data']==dia_indicador, :]

# Faturamento
faturamento_ano = vendas_loja_ano['Valor Final'].sum()
print(f"R${faturamento_ano:,.2f}")
faturamento_dia = vendas_loja_dia['Valor Final'].sum()
print(f"R${faturamento_dia:,.2f}")

# diversidade de produtos
qtd_produtos_ano = len(vendas_loja_ano['Produto'].unique())
print(qtd_produtos_ano)

qtd_produtos_dia = len(vendas_loja_dia['Produto'].unique())
print(qtd_produtos_dia)

# Ticket Médio Anual
valor_venda = vendas_loja_ano.groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()
print(ticket_medio_ano)

# Ticket Médio Dia
valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
print(ticket_medio_dia)

for loja in dicionarios_lojas:

    vendas_loja = dicionarios_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    #print(faturamento_ano)
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    #print(faturamento_dia)

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    #print(qtde_produtos_ano)
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    #print(qtde_produtos_dia)


    #ticket medio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    #print(ticket_medio_ano)
    #ticket_medio_dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    #print(ticket_medio_dia)
    
    
    #enviar o e-mail
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
    #mail.Body = 'Texto do E-mail'
    
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Lira</p>
    '''

    # Anexos (pode colocar quantos quiser):
    attachment  = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()
    print('E-mail da Loja {} enviado'.format(loja))

In [ ]:
# Criar um rank para diretoria

faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_ano)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

In [ ]:
# Enviar o e-mail para diretoria

#enviar o e-mail
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${faturamento_lojas_dia.iloc[0, 0]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${faturamento_lojas_dia.iloc[-1, 0]:.2f}

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano.iloc[0, 0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com Faturamento R${faturamento_lojas_ano.iloc[-1, 0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
Ulysses Alessandro
'''

# Anexos (pode colocar quantos quiser):
attachment  = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment  = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))


mail.Send()
print('E-mail da Diretoria enviado')